In [1]:
import requests
import pandas as pd
import datetime
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
date_from = '2021-08-01'
date_to = str(datetime.date.today())

req_url = f'https://leadgid.api.hasoffers.com/Apiv3/json?api_key=4dc79c63460150a918972cf84c9c4557ff0d063b1252d267fe50f9fcb5abf127&Target=Affiliate_Report&Method=getStats&fields[]=Stat.date&fields[]=Stat.affiliate_info3&fields[]=Stat.affiliate_info2&fields[]=Stat.affiliate_info1&fields[]=Stat.payout&fields[]=Stat.clicks&fields[]=Stat.offer_id&fields[]=Offer.name&filters[Stat.date][conditional]=BETWEEN&filters[Stat.date][values][]={date_from}&filters[Stat.date][values][]={date_to}&limit=5000'
req = requests.get(req_url).json()

ConnectionError: HTTPSConnectionPool(host='leadgid.api.hasoffers.com', port=443): Max retries exceeded with url: /Apiv3/json?api_key=4dc79c63460150a918972cf84c9c4557ff0d063b1252d267fe50f9fcb5abf127&Target=Affiliate_Report&Method=getStats&fields%5B%5D=Stat.date&fields%5B%5D=Stat.affiliate_info3&fields%5B%5D=Stat.affiliate_info2&fields%5B%5D=Stat.affiliate_info1&fields%5B%5D=Stat.payout&fields%5B%5D=Stat.clicks&fields%5B%5D=Stat.offer_id&fields%5B%5D=Offer.name&filters%5BStat.date%5D%5Bconditional%5D=BETWEEN&filters%5BStat.date%5D%5Bvalues%5D%5B%5D=2021-08-01&filters%5BStat.date%5D%5Bvalues%5D%5B%5D=2021-12-08&limit=5000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11e771f40>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
conversion_list = []
columns = ['date', 'offer_name', 'offer_id', 'sub1', 'sub2', 'sub3', 'payout', 'clicks']

for item in req['response']['data']['data']:
    date = item['Stat']['date']
    offer_name = item['Offer']['name']
    offer_id = item['Stat']['offer_id']
    sub1 = item['Stat']['affiliate_info1']
    sub2 = item['Stat']['affiliate_info2']
    sub3 = item['Stat']['affiliate_info3']
    payout = item['Stat']['payout']
    clicks = item['Stat']['clicks']

    conversion_list.append([date, offer_name, offer_id, sub1, sub2, sub3, payout, clicks])

In [ ]:
# conversion_list

In [ ]:
df = pd.DataFrame(columns=columns, data=conversion_list)
df['date'] = pd.to_datetime(df['date'])
df['payout'] = pd.to_numeric(df['payout'])
df['clicks'] = df['clicks'].astype('int')
df['EPC'] = (df['payout'] / df['clicks']).round(2)
df['week'] = df['date'].dt.isocalendar().week
df['month'] = df['date'].dt.month
df.head()

In [ ]:
ppp = df.query('sub1 == "sm" & sub2 == "d0q1"')

In [ ]:
# ppp = ppp.groupby(by=['date', 'offer_id']).agg({'clicks': 'sum', 'payout': 'sum'}).reset_index()
# ppp

ppp = ppp.pivot_table(index='week', columns='sub3', values='payout', aggfunc='sum', fill_value=0)
ppp['total_ckeck'] = ppp['kapusta_check'] + ppp['moneyman_check'] + ppp['vivus_check'] + ppp['webbankir_check']
ppp.tail(10)

In [ ]:
ppp.plot(kind='bar', figsize=(13, 9));

In [ ]:
overall_data = df.pivot_table(index='week', columns='sub1', 
                              values='payout', aggfunc='sum', 
                              fill_value='0', margins=True)

overall_data.sort_values(by='All', axis=1, ascending=False).tail(10)

In [ ]:
ppp = df.query('sub1 == "ppp"').groupby(by=['date', 'offer_id']).agg({'clicks': 'sum', 'payout': 'sum'}).reset_index()
ppp = ppp.pivot_table(index='date', columns='offer_id', values=['clicks', 'payout'], aggfunc='sum', fill_value=0)
ppp.tail()

In [ ]:
# ppp.tail(10).plot(kind='bar', figsize=(13, 9), grid=True);

In [ ]:
# df.query('offer_id == "5141"').max()

In [ ]:
# df.query('offer_id == "5141"').min()